In [1]:
from scraper import Program

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [13]:
# initialize variables to store the data

# pgm_descr = pd.DataFrame(columns=['url', 'header', 'content'])
pgm_other = pd.DataFrame(columns=['url', 'header_tag', 'header_names', 'links_useful_area', 'date_published', 'date_last_modified'])

In [14]:
# load the PPL
ppl = pd.read_excel("../data/PPL/PPL_India.xlsx")
ppl_sub = ppl[['URL', 'Useful area']]
ppl_sub.head()

,URL,Useful area
0,http://cds.iisc.ac.in/admissions/course-degrees/,#page-70
1,https://eecs.iisc.ac.in/mtechai/,#common > div > div > div.col-md-9
2,https://smsuoh.ac.in/mba/mba-business-analytics/,#blog
3,https://scis.uohyd.ac.in/Academic/Programmes/m...,#main-content
4,https://www.iitg.ac.in/ds/,body > main


In [15]:
# change the url, useful_area_path and static variables

problem_rows = []

for row in range(ppl_sub.shape[0]):

    url = ppl_sub['URL'][row]
    uap = ppl_sub['Useful area'][row]

    if '.pdf' in url:
        print(f"{row} PDF document/brochure. No web structure extracted")
        problem_rows.append(row)
        continue

    try:
        p = Program(
            url=url,
            driver_path='../chromedriver_win32/chromedriver.exe',
            useful_area_path=uap,
            static=True
        )

        ex_df = p.extract_df(content=False)
        # concatenate
        pgm_other = pd.concat([pgm_other, ex_df])

    except:
        print(f'Data entry in row {row} needs to be checked')
        problem_rows.append(row)
        continue

Data entry in row 4 needs to be checked
5 PDF document/brochure. No web structure extracted
6 PDF document/brochure. No web structure extracted
7 PDF document/brochure. No web structure extracted
15 PDF document/brochure. No web structure extracted
16 PDF document/brochure. No web structure extracted
17 PDF document/brochure. No web structure extracted
18 PDF document/brochure. No web structure extracted
19 PDF document/brochure. No web structure extracted
20 PDF document/brochure. No web structure extracted
403
Not a successful response
Data entry in row 23 needs to be checked
31 PDF document/brochure. No web structure extracted


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [16]:
ppl_sub.iloc[23]

URL            https://www.amrita.edu/program/m-tech-artifici...
Useful area                                 body > div.container
Name: 23, dtype: object

In [17]:
ppl_sub_fix = ppl_sub.iloc[problem_rows]
ppl_sub_fix.columns = ['url', 'useful_area']

In [18]:
ppl_sub_fix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 4 to 31
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          12 non-null     object
 1   useful_area  2 non-null      object
dtypes: object(2)
memory usage: 288.0+ bytes


In [19]:

pgm_other = pgm_other.append(ppl_sub_fix[['url']]).reset_index(drop=True)

In [20]:
# fix missing dates
pgm_other['date_published'] = pgm_other['date_published'].fillna('Not inferred')
pgm_other['date_last_modified'] = pgm_other['date_last_modified'].fillna('Not inferred')

In [21]:
pgm_other.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   url                 33 non-null     object
 1   header_tag          21 non-null     object
 2   header_names        21 non-null     object
 3   links_useful_area   21 non-null     object
 4   date_published      33 non-null     object
 5   date_last_modified  33 non-null     object
 6   tags                21 non-null     object
 7   tag_freq            21 non-null     object
 8   emphasized          21 non-null     object
dtypes: object(9)
memory usage: 2.4+ KB


In [22]:
# save the intermediate forms of files
pgm_other.to_csv('../data/intermediate/program_other_data_india.csv', index=False)